In [1]:
import bdikit as bdi
import pandas as pd
from IPython.display import display, Markdown
pd.set_option('display.max_rows', None)

In [2]:
df_source = pd.read_csv("./datasets/Dou-ucec-discovery.csv")
df_target = pd.read_csv("./datasets/Dou-ucec-confirmatory.csv")
df_gt = pd.read_csv("./datasets/Dou-ucec-ground-truth.csv")

In [14]:
df_ground_truth = df_gt[~df_gt['target'].isna()]
gt_set = df_ground_truth.set_index('source')['target'].to_dict()
list(gt_set.keys())

['Proteomics_Participant_ID',
 'Case_excluded',
 'Proteomics_TMT_batch',
 'Proteomics_TMT_plex',
 'Proteomics_TMT_channel',
 'Proteomics_Aliquot_ID',
 'Proteomics_Tumor_Normal',
 'Country',
 'Histologic_Grade_FIGO',
 'Myometrial_invasion_Specify',
 'Histologic_type',
 'Treatment_naive',
 'Tumor_purity',
 'Path_Stage_Primary_Tumor-pT',
 'Path_Stage_Reg_Lymph_Nodes-pN',
 'Clin_Stage_Dist_Mets-cM',
 'tumor_Stage-Pathological',
 'BMI',
 'Age',
 'Diabetes',
 'Race',
 'Ethnicity',
 'Gender',
 'Tumor_Site',
 'Tumor_Site_Other',
 'Tumor_Focality',
 'Tumor_Size_cm',
 'Estrogen_Receptor',
 'Progesterone_Receptor',
 'MLH1',
 'MSH6',
 'PMS2',
 'p53',
 'MLH1_Promoter_Hypermethylation',
 'Num_full_term_pregnancies',
 'CIBERSORT_B _cells _naive',
 'CIBERSORT_B _cells _memory',
 'CIBERSORT_Plasma _cells',
 'CIBERSORT_T _cells _CD8',
 'CIBERSORT_T _cells _CD4 _naive',
 'CIBERSORT_T _cells _CD4 _memory _resting',
 'CIBERSORT_T _cells _CD4 _memory _activated',
 'CIBERSORT_T _cells _follicular _helper',
 

In [52]:
def accuracy(gt_set, schema_mapping):
    schema_dict = schema_mapping.set_index('source')['target'].to_dict()
    correct_count = 0
    total = 0
    correct = []
    wrong = []
    for source_column, target_column in  schema_dict.items():
        if source_column in gt_set:
            total += 1
            correct_target_columns = set(gt_set[source_column].split(","))
            if target_column in correct_target_columns:
                correct_count += 1
                correct.append((source_column, target_column))
                # print(f"OK: {source_column} -> out={target_column} gt={correct_target_columns}")
            else:
                wrong.append((source_column, target_column))
                # print(f"ER: {source_column} -> out={target_column} gt={correct_target_columns}")
    return correct_count/float(total), correct, wrong

In [5]:
# df_source.columns = df_source.columns.str.replace(' ', '')

In [53]:
schema_mapping = bdi.match_schema(df_source, df_target, method='ct_learning')
acc, correct, wrong = accuracy(gt_set, schema_mapping)
print(f"{100*acc:.3f}")
schema_mapping.head(10)

Extracting features from 179 columns...


  0%|          | 0/179 [00:00<?, ?it/s]

Extracting features from 213 columns...


  0%|          | 0/213 [00:00<?, ?it/s]

75.556


,source,target
0,idx,xCell_T_cell_CD4+_Th1
1,Proteomics_Participant_ID,Idx
2,Case_excluded,Case_excluded
3,Proteomics_TMT_batch,Metformin_treatment
4,Proteomics_TMT_plex,Number_of_para-aortic_lymph_nodes_positive_for...
5,Proteomics_TMT_channel,Number_of_para-aortic_lymph_nodes_positive_for...
6,Proteomics_Parent_Sample_IDs,Idx
7,Proteomics_Aliquot_ID,Aliquot_ID
8,Proteomics_Tumor_Normal,Group
9,Proteomics_OCT,POLE


In [55]:
schema_mapping = bdi.match_schema(df_source, df_target, method='max_val_sim')
acc, correct, wrong = accuracy(gt_set, schema_mapping)
print(f"{100*acc:.3f}")
schema_mapping.head(10)

Extracting features from 179 columns...


  0%|          | 0/179 [00:00<?, ?it/s]

Extracting features from 213 columns...


  0%|          | 0/213 [00:00<?, ?it/s]

80.000


,source,target
0,idx,Number_of_para-aortic_lymph_nodes_examined
1,Proteomics_Participant_ID,Case_id
2,Case_excluded,Case_excluded
3,Proteomics_TMT_batch,Metformin_treatment
4,Proteomics_TMT_plex,Number_of_para-aortic_lymph_nodes_positive_for...
5,Proteomics_TMT_channel,ReporterName
6,Proteomics_Parent_Sample_IDs,Case_id
7,Proteomics_Aliquot_ID,Aliquot_ID
8,Proteomics_Tumor_Normal,Group
9,Proteomics_OCT,POLE


In [58]:
list(map(lambda x: x[0], correct))

['Proteomics_Participant_ID',
 'Case_excluded',
 'Proteomics_TMT_channel',
 'Proteomics_Aliquot_ID',
 'Proteomics_Tumor_Normal',
 'Country',
 'Histologic_Grade_FIGO',
 'Myometrial_invasion_Specify',
 'Histologic_type',
 'Path_Stage_Primary_Tumor-pT',
 'Path_Stage_Reg_Lymph_Nodes-pN',
 'Clin_Stage_Dist_Mets-cM',
 'tumor_Stage-Pathological',
 'BMI',
 'Age',
 'Diabetes',
 'Race',
 'Ethnicity',
 'Gender',
 'Tumor_Site',
 'Tumor_Site_Other',
 'Tumor_Focality',
 'Tumor_Size_cm',
 'Estrogen_Receptor',
 'Progesterone_Receptor',
 'MLH1',
 'PMS2',
 'p53',
 'MLH1_Promoter_Hypermethylation',
 'Num_full_term_pregnancies',
 'ESTIMATE_StromalScore',
 'ESTIMATE_ESTIMATEScore',
 'CNV_class',
 'MSI_status',
 'POLE_subtype',
 'Genomics_subtype']

In [59]:
df_target['Histologic_grade'].unique()

array(['G1 Well differentiated', 'G2 Moderately differentiated',
       'G3 Poorly differentiated', 'Other: High grade',
       'Other: Not specified.', nan], dtype=object)

In [37]:
schema_mapping = bdi.match_schema(df_source, df_target, method='coma')
print(f"Recall: {100*accuracy(gt_set, schema_mapping):.3f}")
schema_mapping.sample(10)

OK: ESTIMATE_StromalScore -> out=Estimate_StromalScore gt={'Estimate_StromalScore'}
OK: ESTIMATE_ImmuneScore -> out=Estimate_ImmuneScore gt={'Estimate_ImmuneScore'}
OK: Tumor_Site_Other -> out=Tumor_site_other gt={'Tumor_site_other'}
OK: ESTIMATE_ESTIMATEScore -> out=Estimate_ESTIMATEScore gt={'Estimate_ESTIMATEScore'}
OK: Histologic_type -> out=Histologic_Type gt={'Histologic_Type'}
OK: Tumor_Focality -> out=Tumor_focality gt={'Tumor_focality'}
OK: Case_excluded -> out=Case_excluded gt={'Case_excluded'}
OK: Tumor_Size_cm -> out=Tumor_size_cm gt={'Tumor_size_cm'}
OK: Tumor_Site -> out=Tumor_site gt={'Tumor_site'}
OK: MSI_status -> out=MSI_status gt={'MSI_status'}
OK: Ethnicity -> out=Ethnicity gt={'Ethnicity'}
OK: Diabetes -> out=Diabetes gt={'Diabetes'}
OK: Race -> out=Race gt={'Race'}
OK: BMI -> out=BMI gt={'BMI'}
OK: Age -> out=Age gt={'Age'}
OK: CIBERSORT_Eosinophils -> out=Cibersort_Eosinophil gt={'Cibersort_Eosinophil'}
OK: CIBERSORT_Neutrophils -> out=Cibersort_Neutrophil gt={'C

,source,target
90,Pathway_activity_Hypoxia,
136,CNV_AMP,
14,BMI,BMI
152,Log2_SNP_per_Mbp,
109,PI3K_DEPTOR,
141,CNV_4q_DEL,
64,Estrogen_Receptor,
176,miRNAseq_UUID,
150,JAK1_Mutation,
137,CNV_class,


In [60]:
schema_mapping = bdi.match_schema(df_source, df_target, method='similarity_flooding')
print(f"Recall: {100*accuracy(gt_set, schema_mapping):.3f}")
schema_mapping.sample(10)

TypeError: unsupported format string passed to tuple.__format__

In [39]:
schema_mapping = bdi.match_schema(df_source, df_target, method='jaccard_distance')
print(f"Recall: {100*accuracy(gt_set, schema_mapping):.3f}")
schema_mapping.sample(10)

OK: Case_excluded -> out=Case_excluded gt={'Case_excluded'}
OK: Ethnicity -> out=Ethnicity gt={'Ethnicity'}
OK: Gender -> out=Sex gt={'Sex'}
OK: Tumor_Focality -> out=Tumor_focality gt={'Tumor_focality'}
OK: MLH1_Promoter_Hypermethylation -> out=Ancillary_studies_mlh1_promoter_hypermethylation gt={'Ancillary_studies_mlh1_promoter_hypermethylation'}
OK: Num_full_term_pregnancies -> out=Donor_information_number_of_full_term_pregnancies gt={'Donor_information_number_of_full_term_pregnancies'}
OK: MSI_status -> out=MSI_status gt={'MSI_status'}
OK: p53 -> out=Ancillary_studies_p53 gt={'Ancillary_studies_p53'}
OK: Proteomics_Tumor_Normal -> out=Group gt={'Group'}
OK: MLH1 -> out=Ancillary_studies_mlh1 gt={'Ancillary_studies_mlh1'}
OK: Proteomics_TMT_channel -> out=ReporterName gt={'ReporterName'}
OK: Race -> out=Race gt={'Race'}
OK: Clin_Stage_Dist_Mets-cM -> out=Clinical_staging_distant_metastasis_cm gt={'Clinical_staging_distant_metastasis_cm'}
OK: Tumor_Size_cm -> out=Tumor_size_cm gt={'T

,source,target
150,Purity_Cancer,
73,Pathway_activity_Hypoxia,Progeny_NFkB
72,CIBERSORT_Macrophages_M2,Cibersort_Monocyte
177,Methylation_available,
107,ESTIMATE_StromalScore,
43,Progesterone_Receptor,History_of_cancer
22,Tumor_Size_cm,Tumor_size_cm
124,PI3K_PIK3R1,
132,PI3K_RPTOR,
151,Purity_Stroma,


In [17]:
channel_matches = bdi.top_matches(df_source, columns=["Proteomics_TMT_channel"], target=df_target, top_k=3)

Extracting features from 1 columns...


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting features from 213 columns...


  0%|          | 0/213 [00:00<?, ?it/s]

In [18]:
channel_value_matches = bdi.match_values(df_source, df_target, channel_matches, method='tfidf')
bdi.view_value_matches(channel_value_matches)

<br>**Source column:** Proteomics_TMT_channel<br>**Target column:** Pathologic_staging_primary_tumor_pt<br>

BokehModel(combine_events=True, render_bundle={'docs_json': {'5bd1165c-8f4a-4301-9b5f-a6f2b4f0778d': {'version…

<br>**Source column:** Proteomics_TMT_channel<br>**Target column:** Number_of_pelvic_lymph_nodes_examined<br>

BokehModel(combine_events=True, render_bundle={'docs_json': {'44dd946d-f996-4dca-bc64-19306286c802': {'version…

<br>**Source column:** Proteomics_TMT_channel<br>**Target column:** ReporterName<br>

BokehModel(combine_events=True, render_bundle={'docs_json': {'b913b222-2543-4e26-b937-55941670bf40': {'version…

<br>**Source column:** Proteomics_TMT_channel<br>**Target column:** Number_of_para-aortic_lymph_nodes_examined<br>

BokehModel(combine_events=True, render_bundle={'docs_json': {'ce1bd0c8-6918-481f-a64b-f8da82701b9e': {'version…

<br>**Source column:** Proteomics_TMT_channel<br>**Target column:** Number_of_para-aortic_lymph_nodes_positive_for_tumor_by_he<br>

BokehModel(combine_events=True, render_bundle={'docs_json': {'4393b060-5945-49eb-be42-609a15af0837': {'version…

In [33]:
part_id_matches = bdi.top_matches(df_source, columns=["Proteomics_Participant_ID"], target=df_target, top_k=3)
part_id_matches

Extracting features from 1 columns...


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting features from 213 columns...


  0%|          | 0/213 [00:00<?, ?it/s]

,source,target,similarity
0,Proteomics_Participant_ID,Idx,0.824948
1,Proteomics_Participant_ID,Case_id,0.777057
2,Proteomics_Participant_ID,Aliquot_ID,0.616938


In [41]:
bdi.preview_domain(df_source, 'Proteomics_Participant_ID').sample(5)

,value_name
27,C3L-00771
6,C3L-00136
69,C3N-00337
113,NX9
82,C3N-00847


In [46]:
bdi.preview_domain(df_target, 'Idx').head(5)

,value_name
0,C3L-00086
1,C3L-00898
2,C3L-00943
3,C3L-01064
4,C3L-01277


In [47]:
bdi.preview_domain(df_target, 'Case_id').head(5)

,value_name
0,C3L-00086
1,C3L-00898
2,C3L-00943
3,C3L-01064
4,C3L-01277


In [51]:
df_target[(df_target['Idx'] != df_target['Case_id'])]

,Idx,Case_id,Case_excluded,Batch,Plex,ReporterName,Aliquot_ID,Group,Discovery_study,Age,...,Follow-up_additional_surgery_for_new_tumor,Follow-up_additional_treatment_radiation_therapy_for_new_tumor,Follow-up_additional_treatment_pharmaceutical_therapy_for_new_tumor,Follow-up_additional_treatment_immuno_for_new_tumor,Follow-up_days_from_date_of_collection_to_date_of_last_contact,Follow-up_cause_of_death,Follow-up_days_from_date_of_initial_pathologic_diagnosis_to_date_of_death,Follow-up_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor,Follow-up_procedure_type_of_new_tumor,Follow-up_residual_tumor_after_surgery_for_new_tumor
60,C3L-05571-b1,C3L-05571,Yes,b1,4.0,131C,CPT0275270003,Tumor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,C3N-02978-b1,C3N-02978,Yes,b3,12.0,129C,CPT0244470003,Tumor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,C3L-00935-A,C3L-00935,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,C3L-01277-A,C3L-01277,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,C3L-01639-A,C3L-01639,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,C3L-01967-A,C3L-01967,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,C3L-02121-A,C3L-02121,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,C3L-02216-A,C3L-02216,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,C3L-02347-A,C3L-02347,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,C3L-02353-A,C3L-02353,No,NaN,NaN,NaN,NaN,Adjacent_normal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
bdi.preview_domain(df_target, "Cibersort_T_cell_CD8+").head(5)

,value_name
0,0.201397
1,0.059600
2,0.032400
3,0.055500
4,0.107774


In [7]:
bdi.preview_domain(df_source, "Treatment_naive").head(5)

,value_name
0,YES
1,NO
2,NaN


In [10]:
bdi.preview_domain(df_target, "Batch").head(5)

,value_name
0,b4
1,b3
2,b1
3,b2
4,NaN


In [9]:
bdi.match_values(df_source, df_target, ("TP53_TP53", "TP53"), method='tfidf')

,source,target,similarity


In [9]:
bdi.top_matches(df_source, columns=["Proteomics_TMT_batch"], target=df_target, top_k=45)

Extracting features from 1 columns...


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting features from 213 columns...


  0%|          | 0/213 [00:00<?, ?it/s]

,source,target,similarity
0,Proteomics_TMT_batch,Pathologic_staging_primary_tumor_pt,0.408878
1,Proteomics_TMT_batch,Cibersort_T_cell_regulatory_(Tregs),0.373619
2,Proteomics_TMT_batch,ABSOLUTE_tumor_purity,0.363810
3,Proteomics_TMT_batch,Cibersort_T_cell_gamma_delta,0.354802
4,Proteomics_TMT_batch,Metformin_treatment,0.350562
5,Proteomics_TMT_batch,Number_of_para-aortic_lymph_nodes_positive_for...,0.335916
6,Proteomics_TMT_batch,Tumor_stage_pathological,0.308310
7,Proteomics_TMT_batch,Number_of_pelvic_lymph_nodes_positive_for_tumo...,0.301036
8,Proteomics_TMT_batch,TP53,0.284595
9,Proteomics_TMT_batch,baseline_ancillary_studies_other_testing_type_...,0.270495
